In [0]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print("API Key set")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

In [0]:
# TTS

from pydub import AudioSegment
import os
import subprocess
from io import BytesIO
import tempfile

# Set custom temp directory
custom_temp_dir = r"D:\projects\llm_engineering-main\temp"
os.makedirs(custom_temp_dir, exist_ok=True)

# Explicitly set FFmpeg paths
AudioSegment.converter = r"D:\Anaconda3\envs\llms\Library\bin\ffmpeg.exe"
AudioSegment.ffprobe = r"D:\Anaconda3\envs\llms\Library\bin\ffprobe.exe"

def play_audio_with_ffplay(audio_segment, temp_dir):
    # Explicitly create and manage a temporary file
    temp_file_path = os.path.join(temp_dir, "temp_output.wav")
    
    # Export the audio to the temporary file
    audio_segment.export(temp_file_path, format="wav")
    
    try:
        # Play the audio using ffplay
        subprocess.call(["ffplay", "-nodisp", "-autoexit", temp_file_path])
    finally:
        # Clean up the temporary file after playback
        if os.path.exists(temp_file_path):
            os.remove(temp_file_path)

def talker(message):
    # Mocked OpenAI response for testing
    response = openai.audio.speech.create(
        model="tts-1",
        voice="nova",
        input=message
    )
    
    # Handle audio stream
    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")
    
    # Play the audio
    play_audio_with_ffplay(audio, custom_temp_dir)

In [0]:
# STT Whisper

import whisper
import sounddevice as sd
import numpy as np
from scipy.io.wavfile import write

def record_audio(temp_dir, duration=5, samplerate=16000, device_id=2):
    # print(f"Recording for {duration} seconds...")
    sd.default.device = (device_id, None)
    audio = sd.rec(int(duration * samplerate), samplerate=samplerate, channels=1, dtype="int16")
    sd.wait()  # Wait until the recording is finished
    
    audio_path = os.path.join(temp_dir, "mic_input.wav")
    write(audio_path, samplerate, audio)
    # print(f"Audio recorded and saved to {audio_path}")

    return audio_path


whisper_model = whisper.load_model("base")
def transcribe_audio(audio_path):    
    # print("Transcribing audio...")
    result = whisper_model.transcribe(audio_path, language="en")
    return result["text"]

def mic_to_text():
    audio_path = record_audio(custom_temp_dir, duration=10)
    transcription = transcribe_audio(audio_path)
    # print(f"Transcription: {transcription}")
    return transcription

In [0]:
# Chat Functions

import requests
history = [{"role": "system", "content": "You are Nova the friendly robot. Reply within couple of sentences."}]

def run_chat():
    running = True
    while running:
        input_text = input("press Enter to talk")        
        user_input = input_text if input_text.strip() else mic_to_text()
        running = False if input_text == "bye" or user_input.strip() == "bye" else True
        print(f"\nYou: {user_input}\n\n")
        history.append({"role": "user", "content": user_input})    
        api_run = requests.post(
            "http://localhost:11434/api/chat", 
            json={
                "model": "llama3.2",
                "messages": history,
                "stream": False
            }, 
            headers={"Content-Type": "application/json"}
        )
        output_message = api_run.json()['message']['content']
        print(f"Nova: {output_message}\n\n")        
        talker(output_message)
        history.append({"role": "assistant", "content": output_message})

In [0]:
run_chat()